# Carga Histórica - Bronze

In [0]:
from pyspark.sql.functions import split
from pyspark.sql.functions import lit, count
from pyspark.sql.types import StructType, StructField, StringType

In [0]:
# Define o esquema para o arquivo CSV
schema = StructType([
    StructField("YEAR", StringType(), True),
    StructField("MO", StringType(), True),
    StructField("DY", StringType(), True),
    StructField("ALLSKY_SFC_SW_DWN", StringType(), True),
    StructField("CLRSKY_SFC_SW_DWN", StringType(), True),
    StructField("ALLSKY_KT", StringType(), True),
    StructField("ALLSKY_SFC_LW_DWN", StringType(), True),
    StructField("ALLSKY_SFC_PAR_TOT", StringType(), True),
    StructField("CLRSKY_SFC_PAR_TOT", StringType(), True),
    StructField("ALLSKY_SFC_UVA", StringType(), True),
    StructField("ALLSKY_SFC_UVB", StringType(), True),
    StructField("ALLSKY_SFC_UV_INDEX", StringType(), True),
    StructField("T2M", StringType(), True),
    StructField("T2MDEW", StringType(), True),
    StructField("T2MWET", StringType(), True),
    StructField("TS", StringType(), True),
    StructField("T2M_RANGE", StringType(), True),
    StructField("T2M_MAX", StringType(), True),
    StructField("T2M_MIN", StringType(), True)
])

caminho_pasta = '/mnt/projeto_climatico/landing/nasa_solar_flux_temperature/historico/'

# Lista para armazenar os dataframes de cada arquivo CSV
dataframes = []

# Obtém a lista de arquivos no diretório
arquivos = dbutils.fs.ls(caminho_pasta)

# Percorre todos os arquivos no diretório
for arquivo in arquivos:
    if arquivo.name.endswith('.csv'):
        # Extrai o ano do nome do arquivo
        ano = arquivo.name.split('_')[-1].split('.')[0]

        # Lê o arquivo como um RDD de linhas de texto
        caminho_arquivo = arquivo.path
        rdd = spark.sparkContext.textFile(caminho_arquivo)

        # Filtra as linhas após a linha "-END HEADER-"
        header_end_index = rdd.collect().index("-END HEADER-")
        linhas = rdd.collect()[header_end_index + 1:]
        
        # Converte as linhas em um RDD de tuplas
#         rdd_temp = spark.sparkContext.parallelize(linhas).map(lambda line: tuple(line.split(',')))
        
#         df_temp = spark.read.csv(spark.sparkContext.parallelize(linhas), header=True, inferSchema=True, ignoreLeadingWhiteSpace=True)
        df_temp = spark.read.csv(spark.sparkContext.parallelize(linhas), schema=schema, header=True, ignoreLeadingWhiteSpace=True)        
       
        #Converte o RDD em DataFrame
#         df_temp = spark.createDataFrame(rdd_temp)

        # Adiciona uma coluna com o ano
        df_temp = df_temp.withColumn('YEAR', lit(int(ano)))

        # Adiciona o dataframe à lista
        dataframes.append(df_temp)

# Concatena todos os dataframes em um único dataframe
df_final = dataframes[0]
for i in range(1, len(dataframes)):
    df_final = df_final.union(dataframes[i])

df_final.display()

In [0]:
# %sql
# create database bronze

In [0]:
df_final.write.mode("overwrite").option("path", "/mnt/projeto_climatico/bronze/").format("delta").partitionBy("YEAR", "MO").saveAsTable("bronze.nasa_solar_flux_temperature")

In [0]:
%sql
select * from bronze.nasa_solar_flux_temperature
where year = 2022

YEAR,MO,DY,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_KT,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN
2022,1,1,4.3,8.37,0.36,388.82,87.62,160.0,12.55,0.39,1.99,21.51,18.66,20.09,21.9,-265.35,25.87,18.08
2022,1,2,5.42,8.28,0.45,392.36,107.76,159.04,14.95,0.46,2.38,22.64,19.22,20.93,22.65,-266.74,26.08,19.66
2022,1,3,4.57,8.36,0.38,392.23,91.54,161.09,12.83,0.41,2.11,22.95,19.59,21.27,23.36,-264.4,27.97,19.22
2022,1,4,3.51,8.05,0.3,392.39,71.73,155.02,10.16,0.31,1.65,22.8,20.44,21.62,22.99,-264.31,27.55,18.72
2022,1,5,3.98,8.27,0.34,390.41,81.19,159.16,11.73,0.38,1.96,22.41,20.43,21.42,22.5,-268.2,25.19,20.26
2022,1,6,3.91,8.22,0.33,390.33,79.76,158.26,11.56,0.37,1.9,21.05,19.48,20.26,21.22,-267.17,24.76,18.77
2022,1,7,3.07,8.37,0.26,382.62,63.94,160.72,9.48,0.3,1.52,20.07,17.65,18.86,20.62,-266.51,24.45,17.81
2022,1,8,3.42,8.49,0.29,379.76,70.01,162.84,10.16,0.32,1.63,19.55,16.81,18.18,20.4,-264.73,24.64,16.22
2022,1,9,2.41,8.39,0.2,377.0,51.59,160.75,7.84,0.25,1.27,18.51,15.92,17.21,19.19,-266.56,22.34,15.75
2022,1,10,3.75,8.33,0.32,368.73,76.69,158.8,11.1,0.35,1.84,20.28,17.9,19.09,21.03,-264.08,25.77,16.7
